In [20]:
# --- 1. Import Libraries ---
import pandas as pd
import numpy as np

In [21]:
# --- 1. Load Data ---
file_path = "../data/Concert Tour Revenue Dataset.csv"  
df = pd.read_csv(file_path)

In [22]:
# --- 3. Quick Check ---
df.head(3)

,Rank,Peak,All Time Peak,Actual gross,Adjusted gross (in 2022 dollars),Artist,Tour title,Year(s),Shows,Average gross,Ref.
0,1,1,2,"$780,000,000","$780,000,000",Taylor Swift,The Eras Tour †,2023–2024,56,"$13,928,571",[1]
1,2,1,7[2],"$579,800,000","$579,800,000",Beyoncé,Renaissance World Tour,2023,56,"$10,353,571",[3]
2,3,1[4],2[5],"$411,000,000","$560,622,615",Madonna,Sticky & Sweet Tour ‡[4][a],2008–2009,85,"$4,835,294",[6]


Find the literal character "Â" (it often appears due to encoding issues when reading CSVs).

Replace it with an empty string "" (removing it completely).

regex=False tells pandas not to treat "Â" as a regular expression pattern, just match the literal character.

In [23]:
# --- 4. Fix Column Name Encoding ---
df.columns = df.columns.str.replace("Â", "", regex=False).str.strip()

df.columns = (
    df.columns
    .str.replace("\xa0", " ", regex=False)  # replace non-breaking spaces with normal spaces
    .str.strip()  # remove leading/trailing spaces
)

In [24]:
# --- 5. Clean Monetary Columns ---
money_cols = ["Actual gross", "Adjusted gross (2022 USD)", "Average gross"]

for col in money_cols:
    if col in df.columns:
        df[col] = (
            df[col]
            .astype(str)
            .str.replace(r"[\$,]", "", regex=True)
            .str.replace(r"\[.*?\]", "", regex=True)
            .str.strip()
            .replace("", np.nan)
            .astype(float)
        )

In [25]:
# --- 6. Split Year(s) into Start & End Year ---
if "Year(s)" in df.columns:
    df["Start_Year"] = df["Year(s)"].astype(str).str.extract(r"(\d{4})").astype(float)
    df["End_Year"] = df["Year(s)"].astype(str).str.extract(r".*?(\d{4})$").astype(float)

In [26]:
# --- 7. Clean 'Peak' and 'All Time Peak' (remove footnotes like [1]) ---
for col in ["Peak", "All Time Peak"]:
    if col in df.columns:
        df[col] = (
            df[col]
            .astype(str)
            .str.replace(r"\[.*?\]", "", regex=True)
            .replace("nan", np.nan)
            .astype(float)
        )

In [27]:
# --- 8. Trim and Standardize Text Columns ---
text_cols = df.select_dtypes(include="object").columns
for col in text_cols:
    df[col] = df[col].str.strip()

In [28]:
# --- 9. Handle Missing Values ---
df = df.fillna({
    "Peak": -1,
    "All Time Peak": -1
})
df = df.dropna(subset=["Artist", "Tour title"])  # Drop rows missing key info

In [29]:
# --- 10. Remove Duplicates ---
df = df.drop_duplicates()

In [30]:
# --- 11. Save Cleaned Data ---
df.to_csv("../data/concert_tours_clean.csv", index=False)